In [0]:
%run "../Configuration"

In [0]:
# Define the metadata widget:
dbutils.widgets.text("entityName", "")
dbutils.widgets.text("sourceSystem", "")

# Save parameters to variables:
entityName =       dbutils.widgets.get('entityName')
sourceSystemName = dbutils.widgets.get('sourceSystem')

In [0]:
# Define query to extract entityNames from sourceSystem table:
query = f"""
  SELECT entityNames
  FROM   sourceSystem
  WHERE  sourceSystemName = '{sourceSystemName}'
  """

# Query the entity database (query_db).  Find names of entities associated with source system:
entity_names = json.loads(query_entityNames(query))

In [0]:
# For each entity name, create a dataFrame:
for entity in entity_names:
   # Read from BRONZE path in datalake:
   globals()[f"df_{entity}"] = spark.read.format('delta').load(SILVER + entity_silver(entity))

In [0]:
# Iterate through entities to create temp views:
for entity in entity_names:
    # Create or replace tempView:
    globals()[f'df_{entity}'].createOrReplaceTempView(f"tv_{entity}")

In [0]:
if entityName == 'date':
    dbutils.notebook.exit('None')

In [0]:
if entityName == 'design':
    
    # Define Dynamic Query:
    query = f"""
    SELECT design_id
    ,      design_name
    ,      file_location
    ,      file_name
    FROM tv_design
    """
    
    # Execute Query:
    df = spark.sql(query)

In [0]:
if entityName == 'transaction':
    query = f"""
    SELECT transaction_id
    ,      transaction_type
    ,      sales_order_id
    ,      purchase_order_id
    FROM tv_transaction
    """

    # Execute Query:
    df = spark.sql(query)

In [0]:
if entityName == 'payment_type':
    query = f"""
    SELECT payment_type_id
    ,      payment_type_name
    FROM tv_payment_type
    """
    
    # Execute Query:
    df = spark.sql(query)

In [0]:
if entityName == 'location':
    query = f"""
    SELECT address_id AS location_id
    ,      address_line_1
    ,      address_line_2
    ,      district
    ,      city
    ,      postal_code
    ,      country
    ,      phone
    FROM tv_location
    """

    # Execute Query:
    df = spark.sql(query)

In [0]:
if entityName == 'currency':
    query = f"""
    SELECT currency_id
    ,      currency_code
    ,      CASE 
                WHEN currency_code = 'USD' THEN 'United States Dollar'
                WHEN currency_code = 'EUR' THEN 'Euro'
                WHEN currency_code = 'JPY' THEN 'Japanese Yen'
                WHEN currency_code = 'GBP' THEN 'British Pound Sterling'
                WHEN currency_code = 'AUD' THEN 'Australian Dollar'
                WHEN currency_code = 'CAD' THEN 'Canadian Dollar'
                WHEN currency_code = 'CHF' THEN 'Swiss Franc'
                WHEN currency_code = 'CNY' THEN 'Chinese Yuan'
                WHEN currency_code = 'NZD' THEN 'New Zealand Dollar'
                ELSE 'Unknown Currency'
           END AS currency_name   
    FROM tv_currency
    """
    
    # Execute Query:
    df = spark.sql(query)

In [0]:
if entityName == 'counterparty':
    query = f"""
    SELECT cp.counterparty_id
    ,      cp.counterparty_legal_name
    ,      l.address_line_1 AS counterparty_legal_address_line_1
    ,      l.address_line_2 AS counterparty_legal_address_line_2
    ,      l.district AS counterparty_legal_district
    ,      l.city AS counterparty_legal_city
    ,      l.postal_code AS counterparty_legal_postal_code
    ,      l.country AS counterparty_legal_country
    ,      l.phone AS counterparty_legal_phone
    FROM tv_counterparty cp
    JOIN tv_location l ON cp.legal_address_id = l.address_id
    """

    # Execute Query:
    df = spark.sql(query)

In [0]:
if entityName == 'department':
    query = f"""
    SELECT department_id
    ,      department_name
    ,      location AS department_location
    ,      manager AS department_manager
    ,      created_at
    ,      last_updated
    FROM tv_department
    """

    # Execute Query:
    df = spark.sql(query)

In [0]:
if entityName == 'staff':
    query = f"""
    SELECT s.staff_id
    ,      s.first_name
    ,      s.last_name
    ,      d.department_name
    ,      d.location
    ,      s.email_address
    FROM tv_staff s
    JOIN tv_department d ON s.department_id = d.department_id
    """

    # Execute Query:
    df = spark.sql(query)


In [0]:
if entityName == 'payment':
    query = f"""
    SELECT
          ROW_NUMBER() OVER (ORDER BY p.payment_id) AS payment_record_id
        , p.payment_id
        , date_format(p.created_at, 'dd-MM-yy') AS created_date
        , date_format(p.created_at, 'HH:mm:ss') AS created_time
        , date_format(p.last_updated, 'dd-MM-yy') AS last_updated_date
        , date_format(p.last_updated, 'HH:mm:ss') AS last_updated_time
        , t.transaction_id
        , cp.counterparty_id 
        , p.payment_amount
        , c.currency_id
        , pt.payment_type_id
        , p.paid
        , date_format(p.payment_date, 'dd-MM-yy') AS payment_date
    FROM tv_payment p
    JOIN tv_counterparty cp ON p.counterparty_id = cp.counterparty_id
    JOIN tv_currency c ON p.currency_id = c.currency_id
    JOIN tv_payment_type pt ON p.payment_type_id = pt.payment_type_id
    JOIN tv_transaction t ON p.transaction_id = t.transaction_id
    """
 
    # Execute Query:
    df = spark.sql(query)

In [0]:
if entityName == 'purchase_order':
    query = f"""
    SELECT
           ROW_NUMBER() OVER (ORDER BY po.purchase_order_id) as purchase_record_id
    ,      po.purchase_order_id
    ,      date_format(po.created_at, 'dd-MM-yy') AS created_date
    ,      date_format(po.created_at, 'HH:mm:ss') AS created_time
    ,      date_format(po.last_updated, 'dd-MM-yy') AS last_updated_date  
    ,      date_format(po.last_updated, 'HH:mm:ss') AS last_updated_time
    ,      s.staff_id
    ,      cp.counterparty_id
    ,      po.item_code
    ,      po.item_quantity
    ,      po.item_unit_price
    ,      c.currency_id
    ,      po.agreed_delivery_date
    ,      po.agreed_payment_date 
    ,      l.address_id AS agreed_delivery_location_id

    FROM tv_purchase_order po
       JOIN tv_staff        s      ON po.staff_id = s.staff_id 
       JOIN tv_counterparty cp     ON po.counterparty_id = cp.counterparty_id
       JOIN tv_currency     c      ON po.currency_id = c.currency_id
       JOIN tv_location     l      ON po.agreed_delivery_location_id = l.address_id
   
    """

    # Execute Query:
    df = spark.sql(query)



In [0]:
if entityName == 'sales_order':
    query = f"""
    SELECT 
           ROW_NUMBER() OVER (ORDER BY so.sales_order_id) AS sales_record_id
    ,      so.sales_order_id
    ,      date_format(so.created_at, 'dd-MM-yy') AS created_date
    ,      date_format(so.created_at, 'HH:mm:ss') AS created_time
    ,      date_format(so.last_updated, 'dd-MM-yy') AS   last_updated_date
    ,      date_format(so.last_updated, 'HH:mm:ss') AS   last_updated_time
    ,      s.staff_id AS sales_staff_id
    ,      cp.counterparty_id 
    ,      so.units_sold
    ,      so.unit_price
    ,      c.currency_id
    ,      d.design_id
    ,      so.agreed_payment_date
    ,      so.agreed_delivery_date
    ,      l.address_id AS agreed_delivery_location_id
    FROM tv_sales_order so
    JOIN tv_counterparty cp ON so.counterparty_id = cp.counterparty_id
    JOIN tv_staff s ON so.staff_id = s.staff_id
    JOIN tv_currency c ON so.currency_id = c.currency_id
    JOIN tv_design d ON so.design_id = d.design_id
    JOIN tv_location l on so.agreed_delivery_location_id = l.address_id
    """
    
    # Execute Query:
    df = spark.sql(query)

In [0]:
# Check data exists:
if df.count() > 0:
    # Set location for delta table:
    file_path = f"/{sourceSystemName}/{entityName}"
    
    # Write to delta table:
    df.write.mode('overwrite').format('delta').option('header', 'true').save(GOLD + file_path)

    # Output file_path:
    dbutils.notebook.exit(file_path)
